In [1]:
data_file = '/notebooks/ratings.dat'

In [18]:
!nvidia-smi

Sun Apr 11 15:22:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:3F:00.0 Off |                  N/A |
| 28%   32C    P2    35W / 250W |  11962MiB / 12066MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
data_file

'/notebooks/ratings.dat'

In [4]:
lines = open(data_file)

In [5]:
for i, line in enumerate(lines):
    if i >= 10:
        break
    print(line.strip())

1::122::5::838985046
1::185::5::838983525
1::231::5::838983392
1::292::5::838983421
1::316::5::838983392
1::329::5::838983392
1::355::5::838984474
1::356::5::838983653
1::362::5::838984885
1::364::5::838983707


In [6]:
count = 0
for line in lines:
    count += 1
print("总共数量：", count)

总共数量： 10000043


原始数据格式
------------------------
UserId::MovieId::Score::Ts


要求：
------------------------
1、抽取用户行为数小于2000的用户

2、基于时间戳排序后，划分训练样本（取后2个行为作为测试样本test_set）

3、对数据只取userId、movieId、Rating

In [7]:
# step01:读取数据，读取数据的最后格式为 {uid: [(mid, score, ts)]}
def read_raw_data(file_path):
    user_info = dict()
    lines = open(file_path)
    for line in lines:
        tmp = line.strip().split("::")
        if len(tmp) < 4:
            continue
        ui = user_info.get(tmp[0], None)
        if ui is None:
            user_info[tmp[0]] = [(tmp[1], tmp[2], tmp[3])]
        else:
            user_info[tmp[0]].append((tmp[1], tmp[2], tmp[3]))
    
    return user_info

In [8]:
user_info = read_raw_data(data_file)

In [9]:
# 简单统计下每个用户的行为数，基于行为数过滤掉一些异常用户
user_action_num = {}
for k,v in user_info.items():
    user_action_num[k] = len(v)
    

#
import numpy as np
user_stat = np.asarray(list(user_action_num.values()))
print("总共的用户量：", len(user_stat))
max_num = np.max(user_stat)
min_num = np.min(user_stat)
median_num = np.median(user_stat)
average_num = np.average(user_stat)

print(max_num, min_num, median_num, average_num)

总共的用户量： 69878
7359 20 69.0 143.10732991785684


In [10]:
filter_user_nums = 0
for n in user_stat:
    if n > 2000:
        filter_user_nums += 1
print(filter_user_nums)

97


In [11]:
# 抽取用户行为数小于2000的用户
def extract_vaild_user(user_info):
    user_info_filter = {}
    for k, v in user_info.items():
        if len(v) > 2000:
            continue
        user_info_filter[k] = v
    return user_info_filter

user_info = extract_vaild_user(user_info)

In [12]:
print(len(user_info))

69781


In [13]:
# step03：基于时间戳对行为序列进行排序，并划分为训练集和测试集
def split_train_test(user_info):
    train_set = []
    test_set = []
    for k, v in user_info.items():
        tmp = sorted(v, key=lambda _:_[2]) # 基于时间排序
        for i in range(len(tmp)):
            if i < len(tmp) - 2:
                train_set.append(str(k) + "," + tmp[i][0] + "," + tmp[i][1])
            else:
                test_set.append(str(k) + "," + tmp[i][0] + "," + tmp[i][1])
    return train_set, test_set

train_set, test_set = split_train_test(user_info)

In [14]:
# step04: 将训练数据和验证数据保存下来
def save_data(train_set, test_set, save_path_dir):
    import random
    random.shuffle(train_set)
    random.shuffle(test_set)
    with open(save_path_dir + "train_set", "w") as f:
        for line in train_set:
            f.write(line + "\n")
    
    with open(save_path_dir + "test_set", "w") as f:
        for line in train_set:
            f.write(line + "\n")
            
save_path = ""
save_data(train_set, test_set, save_path)

转换数据格式-hash化：
------------------------
1、对数据中特征进行hash化处理

2、特征hash化的逻辑：hash_fun(特征域=特征值)；Hash("user_id=1")/Hash("movie_id=1")

3、hash离散化的作用哦：a.更好的稀疏表示；b.具有一定的特征压缩；c.能够方便模型的部署

In [15]:
# 定义hash化方法
def bkdr2hash64(str01):
    mask60 = 0x0ffffffffffffff
    seed = 131
    hash = 0
    for s in str01:
        hash = hash * seed + ord(s)
    return hash & mask60

In [16]:
print("用户id", bkdr2hash64("UserID=1"))
print("电影id", bkdr2hash64("MovieID=1"))

用户id 56860453139892626
电影id 50958815124388695


In [17]:
def tohash(file, save_path):
    wfile = open(save_path, "w")
    with open(file) as f:
        for line in f:
            tmp = line.strip().split(",")
            user_id = bkdr2hash64("UserID="+tmp[0])
            item_id = bkdr2hash64("MovieID="+tmp[1])
            wfile.write(str(user_id) + "," + str(item_id) + "," + tmp[2] + "\n")
    wfile.close()
    
train_file_path = "./save_data/train_set"
train_tohash = "./save_data/train_set_tohash"
test_file_path = "./save_data/test_set"
test_tohash = "./save_data/test_set_tohash"
tohash(train_file_path, train_tohash)
tohash(test_file_path, test_tohash)